**all neccessary library imports**

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from time import time
import pickle

**import dataset and clean up data**

In [18]:
query_data = pd.read_csv("./data/sqli.csv", encoding='utf-8')

# remove duplicates from dataset
query_data.drop_duplicates(inplace=True)

# handle any missing value on dataset
query_data.dropna(inplace=True)
print(f"Data shape: {query_data.shape}")
query_data.head(10)

Data shape: (30907, 2)


,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
6,select * from users where id = 1 +$+ or 1 =...,1
7,"1; ( load_file ( char ( 47,101,116,99,47...",1
8,select * from users where id = '1' or ||/1 ...,1
9,select * from users where id = '1' or \.<\ ...,1


In [19]:
X = query_data['Query']
y = query_data['Label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1) 
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [5]:
print(f"X_train : {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

X_train : (21634, 18698)
y_train: (21634,)
X_test: (9273, 18698)
y_test: (9273,)


In [22]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
X_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

> test different model

In [6]:
# training a model using decision tree algorithm
d_model = DecisionTreeClassifier()
start_time = time()
d_model.fit(X_train, y_train)
end_time = time()
d_pred = d_model.predict(X_test)
d_model_accuracy = accuracy_score(y_test, d_pred)
d_model_f1_score = f1_score(y_test, d_pred)
print(f"Accuracy: {d_model_accuracy}")
print(f"F1-Score: {d_model_f1_score}")
print(f"Training Time: {end_time-start_time:.2f} seconds")

Accuracy: 0.7960746252561199
F1-Score: 0.7817153411058524
Training Time: 7.56 seconds


In [7]:
# training a model using random forest tree algorithm
r_model = RandomForestClassifier()
start_time = time()
r_model.fit(X_train, y_train)
end_time = time()
r_pred = r_model.predict(X_test)
r_model_accuracy = accuracy_score(y_test, r_pred)
r_model_f1_score = f1_score(y_test, r_pred)
print(f"Accuracy: {r_model_accuracy}")
print(f"F1-Score: {r_model_f1_score}")
print(f"Training Time: {end_time-start_time:.2f} seconds")

Accuracy: 0.8010352636687156
F1-Score: 0.790174002047083
Training Time: 14.65 seconds


In [8]:
# training a model using Support Vector Machine (SVM) algorithm
s_model = SVC()
start_time = time()
s_model.fit(X_train, y_train)
end_time = time()
s_pred = s_model.predict(X_test)
s_model_accuracy = accuracy_score(y_test, s_pred)
s_model_f1_score = f1_score(y_test, s_pred)
print(f"Accuracy: {s_model_accuracy}")
print(f"F1-Score: {s_model_f1_score}")
print(f"Training Time: {end_time-start_time:.2f} seconds")

Accuracy: 0.8208778173190985
F1-Score: 0.8034552124008992
Training Time: 72.19 seconds


> chosen naive bayes model

In [24]:
# training a naive Bayes model
n_model = MultinomialNB()
start_time = time()
n_model.fit(X_train,y_train)
end_time = time()
n_pred = n_model.predict(X_test)
n_model_accuracy = accuracy_score(y_test, n_pred)
n_model_f1_score = f1_score(y_test, n_pred)
print(f"Accuracy: {n_model_accuracy}")
print(f"F1-Score: {n_model_f1_score}")
print(f"Training Time: {end_time-start_time:.2f} seconds")

Accuracy: 0.9716380890758115
F1-Score: 0.9618564176939812
Training Time: 0.01 seconds


In [10]:
# save this training model for futher use
pickle.dump(n_model, open("model.pkl", "wb"))

In [11]:
# load model and test 
model = pickle.load(open("model.pkl", "rb"))

with open("sql_query.txt", "r") as file:
    lines = file.readlines()
    
user_given_query_data = pd.DataFrame(lines, columns=['Query'])

sql_query = vectorizer.transform(user_given_query_data['Query'])
results = model.predict(sql_query)

malicious = 0
safe = 0
for result in results:
    if result == 1:
        malicious = malicious + 1
    else:
        safe = safe + 1
        
print(f"Flag out malicious queries as malicious: {malicious}")
print(f"Flag out malicious queries as safe: {safe}")

Flag out malicious queries as malicious: 97
Flag out malicious queries as safe: 3


#### manual testing
Perform a manual testing on a website that has input validation with these same 100 malicious queries.

> Result of manual testing

```
Flag out malicious queries as malicious : 78
Flag out malicious queries as safe: 22
```